In [1]:
import bempp.api
from bempp.api.assembly.blocked_operator import BlockedOperator
import math
import numpy as np
import scipy 
import cmath
from numba import objmode
from numpy.linalg import slogdet
import matplotlib.pyplot as plt

In [2]:
length_ = 5 # length of each screen
width_ = 7  # width of each screen
dist_ = 0.5 # distance between the screens
h = 0.1 # grid size

In [3]:
corners1 = np.array([ [-0.5*width_, -0.5*length_, 0.5*dist_],
                     [-0.5*width_,  0.5*length_, 0.5*dist_],
                     [0.5*width_,   0.5*length_, 0.5*dist_],
                     [0.5*width_,  -0.5*length_, 0.5*dist_]])
corners2 = np.array([ [-0.5*width_, -0.5*length_, -0.5*dist_],
                     [-0.5*width_,  0.5*length_, -0.5*dist_],
                     [0.5*width_,   0.5*length_, -0.5*dist_],
                     [0.5*width_,  -0.5*length_, -0.5*dist_]])
grid1 = bempp.api.shapes.screen(corners1, h = h)
grid2 = bempp.api.shapes.screen(corners2, h = h)

In [4]:
space1 = bempp.api.function_space(grid1, "P", 1)
space2 = bempp.api.function_space(grid2, "P", 1)

In [5]:
N_wn = 15
lbound_wn = 0
ubound_wn = 4
quad_points = np.linspace(np.exp(-ubound_wn), np.exp(-lbound_wn), N_wn)
results = np.empty(N_wn, dtype='float64')

In [ ]:
for index, point in enumerate(quad_points):

    wavenumber = -1j * np.log(point)

    slp11 = bempp.api.operators.boundary.helmholtz.single_layer(space1, space1, space1, wavenumber)
    slp12 = bempp.api.operators.boundary.helmholtz.single_layer(space2, space1, space1, wavenumber)
    slp21 = bempp.api.operators.boundary.helmholtz.single_layer(space1, space2, space2, wavenumber)
    slp22 = bempp.api.operators.boundary.helmholtz.single_layer(space2, space2, space2, wavenumber)

    mat11 = slp11.weak_form().A
    mat12 = slp12.weak_form().A
    mat21 = slp21.weak_form().A
    mat22 = slp22.weak_form().A
    mat12_zero = np.zeros((mat11.shape[0],mat12.shape[1]))
    mat21_zero = np.zeros((mat22.shape[0],mat11.shape[1]))

    mat = np.block([[mat11,mat12],[mat21,mat22]])  
    mat2 = np.block([[mat11,mat12_zero],[mat21_zero,mat22]])
    
    n = mat.shape[0]
    Id = np.eye(n)
    inv_mat2 = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat2), Id)

    combined = inv_mat2.dot(mat)
    sign, logdet = slogdet(combined)
    
    results[index] = logdet
    print(f"Log Determinant: {index}: {logdet}")

Log Determinant: 0: -0.5029392070820645
Log Determinant: 1: -1.7149526049284827
Log Determinant: 2: -2.6141101097770734
Log Determinant: 3: -3.3604554234714485
Log Determinant: 4: -4.004166965075259
Log Determinant: 5: -4.570039197905504
Log Determinant: 6: -5.072624867870161
Log Determinant: 7: -5.521262096350433
Log Determinant: 8: -5.922220499877856


In [ ]:
%matplotlib inline
from matplotlib.pyplot import figure

fig = figure()
ax1 = fig.add_subplot(121)
ax1.set_xlabel('1j * k')
ax2 = fig.add_subplot(122)
ax2.set_xlabel('exp(-k)')
ax1.semilogy(-np.log(quad_points), -results)
ax2.plot(quad_points, -results)
fig.tight_layout()

In [ ]:
casimir_energy = np.trapz(-results / quad_points, quad_points) / (2 * np.pi)
print(f"The normalized Casimir Energy is: {casimir_energy}")

area_ = length_ * width_
casimir_energy_unit_area = casimir_energy/area_
print(f"The normalized Casimir Energy per unit area is: {casimir_energy_unit_area}")

The following paper states the Casimir energy between two perfectly conducting plates at a distance $a$ is:

$$\mathcal{E}_{\text{per unit area}}(a) = \frac{1}{2}\frac{\pi^{2}}{720}\frac{1}{a^{3}}$$

https://iopscience.iop.org/article/10.1088/1126-6708/2003/06/018/pdf?casa_token=lsiQlkbo0DQAAAAA:sDu0bRRkVhtihAFr3TrMvPw5yHmJeTAKXwWN59jyzKqZqh0PCE1-QuGkebrkYN13rHwq40nP

In [ ]:
ref_ = 0.5 * (np.pi**2) /(720*dist_**3)
print(casimir_energy_unit_area, ref_)

In [ ]:
X = np.array([[9, 3, 1, 5], [3, 7, 5, 1], [1, 5, 9, 2], [5, 1, 2, 6]])
p = X.shape[0]
Ip = np.eye(p)
scipy.linalg.lu_solve(scipy.linalg.lu_factor(X), Ip) @ X